# Linear Regression Cruise Crew Prediction

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('CruiseCrewPrediction').getOrCreate()

In [2]:
data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [3]:
data.printSchema()
print(data.head())
data.columns

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)


['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [4]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors

In [5]:
index = StringIndexer(
    inputCol = "Cruise_line",
    outputCol = "CruiseLine")

indexer = index.fit(data).transform(data)

In [6]:
indexer.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- CruiseLine: double (nullable = true)



In [7]:
indexer.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'CruiseLine']

In [8]:
assembler = VectorAssembler(
    inputCols = ['passengers', 'length',
                 'cabins', 'passenger_density', 'CruiseLine'],
    outputCol = "features")

output = assembler.transform(indexer)
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- CruiseLine: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
mldata = output.select('features','crew')
train,test = mldata.randomSplit([0.7,0.3])
unlabelled = test.select('features')

In [10]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol='crew', featuresCol='features')
lrModel = lr.fit(train)

In [11]:
print("Intercept :{}".format(lrModel.intercept))
print("Coefficients :{}".format(lrModel.coefficients))

Intercept :-1.528136820836566
Coefficients :[-0.157457467879,0.374838482074,0.979094403977,0.00904906200921,0.0425760104199]


In [12]:
#testing the model on the test data set

test_results = lrModel.evaluate(test)

In [13]:
#Printing the insights that we receive from the testing of the model

print("R-Squared :{}".format(test_results.r2))
print("RMSE :{}".format(test_results.rootMeanSquaredError))

R-Squared :0.951897457200906
RMSE :0.8054347265532542


In [14]:
#deploying our model on the unlabelled data
deployresult = lrModel.transform(unlabelled)

In [15]:
deployresult.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[0.66,2.8,0.33,50...|0.5392617557479469|
|[2.08,4.4,1.04,48...|1.8004661840974954|
|[2.08,4.4,1.04,48...|1.8004661840974954|
|[2.96,5.14,1.48,5...|2.3634794642487327|
|[3.82,5.97,1.94,6...| 3.068201756683557|
|[3.88,5.97,1.94,6...|3.0495242653614083|
|[6.84,5.94,3.42,4...| 3.881321153556912|
|[6.86,5.93,3.44,4...|3.4671595158184783|
|[6.94,5.94,3.55,4...| 4.148502240510507|
|[7.0,7.09,3.54,71...| 4.565394535509171|
|[8.08,6.16,4.1,35...| 4.054215488784876|
|[8.26,5.78,4.25,2...| 4.549807763255093|
|[9.52,5.41,3.83,1...| 3.208876310708116|
|[12.64,7.19,6.32,...|5.8892764860213545|
|[12.66,7.19,6.33,...|5.8952848463628955|
|[13.66,6.82,6.87,...| 6.172053307178485|
|[14.4,7.77,7.2,43...| 6.690074850215622|
|[14.94,7.98,7.47,...|  6.87853210101937|
|[14.94,7.98,7.67,...| 7.159503002654497|
|[15.04,7.08,7.52,...| 6.543393576848666|
+--------------------+------------